In [1]:
%load_ext sql
import os
from sqlalchemy import create_engine

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)
conn = create_engine(dsl)

# MagicコマンドでSQLを書くための設定
%sql conn

In [12]:
%%sql
drop table if exists kaiin;
CREATE TABLE kaiin (
    行番号 INTEGER PRIMARY KEY,
    会員番号 TEXT,
    項目名 TEXT,
    項目値 TEXT
);
INSERT INTO kaiin (行番号, 会員番号, 項目名, 項目値) VALUES
(1, '0111', '会員名', '情報太郎'),
(2, '0111', '最終購入年月日', '2019-02-05'),
(3, '0112', '会員名', '情報花子'),
(4, '0112', '最終購入年月日', '2019-01-30'),
(5, '0112', '最終購入年月日', '2019-02-01'),
(6, '0113', '会員名', '情報次郎');

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.
6 rows affected.


[]

In [13]:
%%sql
select * from kaiin;

*  postgresql://padawan:***@db:5432/dsdojo_db
6 rows affected.


行番号,会員番号,項目名,項目値
1,0111,会員名,情報太郎
2,0111,最終購入年月日,2019-02-05
3,0112,会員名,情報花子
4,0112,最終購入年月日,2019-01-30
5,0112,最終購入年月日,2019-02-01
6,0113,会員名,情報次郎


### Q: テーブルの条件のもと、以下の結果を得るために必要なSQLは？

条件
- 会員ごとに固有の会員番号を持つ
- 同一会員番号で、同一項目名の行が複数ある場合、より大きい行番号の項目値を採用する

| 会員番号 | 会員名 | 最終購入年月日 |
| :---: | :---: | :---: |
| 0111 | 情報太郎 | 2019-02-05 |
| 0112 | 情報花子 | 2019-02-01 |
| 0113 | 情報次郎 | NULL |

In [ ]:
%%sql
-- A. 会員ごとに最新の会員名、最終購入年月日を取得
SELECT
    t1.会員番号,
    MAX(CASE WHEN t1.項目名 = '会員名' THEN t1.項目値 END) AS 会員名,
    MAX(CASE WHEN t1.項目名 = '最終購入年月日' THEN t1.項目値 END) AS 最終購入年月日
FROM (
    SELECT
        t2.会員番号,
        t2.項目名,
        t2.項目値
    FROM
        kaiin t2
    WHERE
        t2.行番号 IN (
            SELECT
                MAX(行番号)
            FROM
                kaiin
            GROUP BY
                会員番号, 項目名
        )
) AS t1
GROUP BY
    t1.会員番号
ORDER BY
    t1.会員番号;

*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


会員番号,会員名,最終購入年月日
0111,情報太郎,2019-02-05
0112,情報花子,2019-02-01
0113,情報次郎,None
